
# AMSter — Sentinel‑1 Bologna Subset (Time‑Series Quickstart, HPC‑ready)

This notebook is **AMSter‑only** (no ISCE/GMTSAR/MintPy vars).  
It sets up environment checks, pairs a small Sentinel‑1 SLC subset, and runs a minimal time‑series chain with basic benchmarking.

**Assumptions**
- You already installed AMSter via the installer.
- Your subset root is: `/mnt/data/slc_stack_test_subset` (adjust if needed).
- Orbits, DEMs & masks are on your shared cluster mount under `/mnt/data/DataSAR` (as configured during install).


In [3]:

# --- 1) Define your working subset paths ---
WORK_DIR = "home/ubuntu/work//slc_stack_test_subset"  # <- change if your subset lives elsewhere
SLC_DIR  = f"{WORK_DIR}/data/slcs"
AUX_ORB  = f"{WORK_DIR}/data/aux/orbits"
DEM_DIR  = f"{WORK_DIR}/data/dem"
OUT_DIR  = f"{WORK_DIR}/results_amster"

print("WORK_DIR:", WORK_DIR)
print("SLC_DIR :", SLC_DIR)
print("AUX_ORB :", AUX_ORB)
print("DEM_DIR :", DEM_DIR)
print("OUT_DIR :", OUT_DIR)


WORK_DIR: home/ubuntu/work//slc_stack_test_subset
SLC_DIR : home/ubuntu/work//slc_stack_test_subset/data/slcs
AUX_ORB : home/ubuntu/work//slc_stack_test_subset/data/aux/orbits
DEM_DIR : home/ubuntu/work//slc_stack_test_subset/data/dem
OUT_DIR : home/ubuntu/work//slc_stack_test_subset/results_amster


In [8]:
from pathlib import Path
import os

dirs_to_check = [
  WORK_DIR,
  "/mnt/data/DataSAR",
  os.environ.get("PATH_DataSAR", ""),
  os.environ.get("EXTERNAL_DEMS_DIR", ""),
  os.environ.get("EXTERNAL_MASKS_DIR", "")
]

print("Checking directories exist:")
for d in dirs_to_check:
  if d and Path(d).is_dir():
    print(f"  OK      {d}")
  else:
    print(f"  MISSING {d}")

print("\nListing a few SLC SAFE folders:")
slc_dir = Path("/home/ubuntu/work/slc_stack_test_subset/data/slcs")
if slc_dir.is_dir():
  for i, f in enumerate(sorted(slc_dir.iterdir()), 1):
    if i > 8:
      break
    print(f.name)
else:
  print("(none found)")

print("\nListing local subset orbits:")
orbit_dir = Path("/home/ubuntu/work/slc_stack_test_subset/data/aux/orbits")
if orbit_dir.is_dir():
  files = list(sorted(orbit_dir.iterdir()))
  if files:
    for f in files[:8]:
      print(f.name)
  else:
    print("(none here)")
else:
  print("(none here)")



Checking directories exist:
  MISSING home/ubuntu/work//slc_stack_test_subset
  OK      /mnt/data/DataSAR
  MISSING 
  MISSING 
  MISSING 

Listing a few SLC SAFE folders:
S1A_IW_SLC__1SDV_20250310T113934_20250310T114001_058242_07329D_35D1.zip
S1A_IW_SLC__1SDV_20250315T114721_20250315T114748_058315_073579_BEBC.zip
S1A_IW_SLC__1SDV_20250315T114746_20250315T114813_058315_073579_B6C8.zip
S1A_IW_SLC__1SDV_20250322T113935_20250322T114002_058417_073988_9F68.zip
S1A_IW_SLC__1SDV_20250324T233147_20250324T233214_058453_073AF4_BD46.zip
S1A_IW_SLC__1SDV_20250324T233212_20250324T233239_058453_073AF4_1194.zip
S1A_IW_SLC__1SDV_20250327T114721_20250327T114748_058490_073C5E_B388.zip
S1A_IW_SLC__1SDV_20250327T114746_20250327T114813_058490_073C5E_FF9E.zip

Listing local subset orbits:
(none here)



### Orbit directory preference
If your global `$S1_ORBITS_DIR` is empty but your subset already contains precise orbits under `data/aux/orbits`, we set a **temporary session override** (does NOT modify `~/.bashrc`).


In [9]:

# --- 3) Orbit directory override if helpful ---
from pathlib import Path
import os

subset_orbits = Path(AUX_ORB)
global_orbits = os.environ.get("S1_ORBITS_DIR", "")
needs_override = False

if subset_orbits.exists() and any(subset_orbits.iterdir()):
    # If global orbits dir seems empty or undefined, prefer the subset
    if not global_orbits or not Path(global_orbits).exists() or not any(Path(global_orbits).iterdir()):
        os.environ["S1_ORBITS_DIR"] = str(subset_orbits)
        needs_override = True

print("Using S1_ORBITS_DIR =", os.environ.get("S1_ORBITS_DIR", "(unset)"))
print("Override applied in this notebook session only:", needs_override)


Using S1_ORBITS_DIR = (unset)
Override applied in this notebook session only: False


In [ ]:

# --- 4) (Optional) Download a few days of S1 precise orbits to the global orbit dir ---
# Adjust dates to your subset; harmless if files already exist.
%bash -s << 'EOF'
set -euo pipefail
source ~/.bashrc
if command -v downloadPreciseOrbits >/dev/null 2>&1; then
  echo "Downloading sample orbits S1A 2023-05-01..2023-05-03 into $S1_ORBITS_DIR"
  downloadPreciseOrbits S1A 20230501 20230503 || true
else
  echo "downloadPreciseOrbits not found; you can skip or run from AMSTerEngine folder."
fi
EOF


In [ ]:

# --- 5) Create output folder ---
%bash -s << 'EOF'
set -euo pipefail
OUT_DIR="/mnt/data/slc_stack_test_subset/results_amster"
mkdir -p "$OUT_DIR"
echo "Created: $OUT_DIR"
EOF


In [ ]:

# --- 6) Detect available AMSter commands (engine + scripts) ---
%bash -s << 'EOF'
set -euo pipefail
source ~/.bashrc
echo "Engine binaries present:"
if [ -d "/home/ubuntu/SAR/AMSTer/AMSTerEngine" ]; then
  (cd /home/ubuntu/SAR/AMSTer/AMSTerEngine && ls -1 | head -n 40)
else
  echo "AMSTerEngine folder not found at /home/ubuntu/SAR/AMSTer/AMSTerEngine"
fi

echo
echo "High-level scripts under PATH_SCRIPTS:"
if [ -n "$PATH_SCRIPTS" ] && [ -d "$PATH_SCRIPTS" ]; then
  ls -1 "$PATH_SCRIPTS" | head -n 40
  echo
  echo "Common entrypoints (if present):"
  for s in AMSTer_StackProcessing.sh AMSTer_Amplitude.sh S1_pairs_generation.sh; do
    if [ -f "$PATH_SCRIPTS/$s" ]; then echo "  FOUND  $s"; else echo "  missing $s"; fi
  done
else
  echo "PATH_SCRIPTS not set or directory missing."
fi
EOF



### AMSter minimal config (template)

We write a **config file** in your project to keep things reproducible.  
If your distribution expects differently named keys, edit in-place and re-run.


In [ ]:

# --- 7) Write a minimal AMSter config template ---
from pathlib import Path
cfg_path = Path("/mnt/data/slc_stack_test_subset/AMSTer.cfg")
cfg_text = f"""
# ===== AMSter Minimal Config (edit as needed) =====
PROJECT_NAME = Bologna_S1_Test
MISSION      = S1
STACK_MODE   = SBAS         # SBAS | INTERF | AMPLITUDE
NB_PROC      = 8

# --- paths ---
SLC_DIR   = /mnt/data/slc_stack_test_subset/data/slcs
WORK_DIR  = /mnt/data/slc_stack_test_subset/results_amster
ORBIT_DIR = {os.environ.get('S1_ORBITS_DIR','/mnt/data/DataSAR/SAR_AUX_FILES/ORBITS/S1_ORB')}
DEM_DIR   = /mnt/data/slc_stack_test_subset/data/dem

# Optional constraints for pair selection (adjust to your AOI):
# TBASE_MAX_DAYS = 48
# BPERP_MAX_M    = 150

# Optional: master date (yyyymmdd); if empty, AMSter picks automatically
# MASTER_DATE = 

# ===== End of file =====
"""
cfg_path.write_text(cfg_text)
print("Wrote", cfg_path)
print(cfg_text)


In [ ]:

# --- 8) Generate pairs (if script exists) ---
%bash -s << 'EOF'
set -euo pipefail
source ~/.bashrc
CFG="/mnt/data/slc_stack_test_subset/AMSTer.cfg"
if [ -f "$PATH_SCRIPTS/S1_pairs_generation.sh" ]; then
  echo "Running pair generation..."
  bash "$PATH_SCRIPTS/S1_pairs_generation.sh" "/mnt/data/slc_stack_test_subset/data/slcs" || true
else
  echo "S1_pairs_generation.sh not found — skipping (pairs may be auto-generated by the stack script)."
fi
EOF


In [ ]:

# --- 9) Run stack processing (time + log for benchmarking) ---
# This uses /usr/bin/time -v to capture CPU/memory; adjust if not available on your cluster.
%bash -s << 'EOF'
set -euo pipefail
source ~/.bashrc
CFG="/mnt/data/slc_stack_test_subset/AMSTer.cfg"
LOG="/mnt/data/slc_stack_test_subset/results_amster/run_time.log"

if [ -f "$PATH_SCRIPTS/AMSTer_StackProcessing.sh" ]; then
  echo "Launching AMSter stack processing with benchmarking..."
  /usr/bin/time -v bash "$PATH_SCRIPTS/AMSTer_StackProcessing.sh" "$CFG" 2>&1 | tee "$LOG"
else
  echo "AMSTer_StackProcessing.sh not found; try running engine-level steps or check PATH_SCRIPTS."
fi
EOF


In [ ]:

# --- 10) Visualize outputs if present (generic search for GeoTIFFs) ---
import os, glob
import matplotlib.pyplot as plt
from osgeo import gdal

search_roots = [
    "/mnt/data/slc_stack_test_subset/results_amster",
    "/mnt/data/slc_stack_test_subset"
]

candidates = []
for root in search_roots:
    for pat in ("**/*time*series*.tif", "**/*deform*.tif", "**/*veloc*.tif", "**/*coher*.tif", "**/*unw*.tif"):
        candidates.extend(glob.glob(os.path.join(root, pat), recursive=True))

print("Found raster candidates:", len(candidates))
for i, p in enumerate(candidates[:5], 1):
    print(f"{i}. {p}")

# Quicklook the first one, if any
if candidates:
    ds = gdal.Open(candidates[0])
    arr = ds.ReadAsArray()
    if arr.ndim == 3:
        img = arr[-1]  # last epoch if time stack
    else:
        img = arr
    plt.figure(figsize=(7,6))
    plt.imshow(img, cmap="gray")
    plt.title(os.path.basename(candidates[0]))
    plt.colorbar()
    plt.show()
else:
    print("No GeoTIFF outputs detected yet. Check logs above.")


In [ ]:

# --- 11) Parse benchmark log (if created) ---
import re, pathlib
log_path = pathlib.Path("/mnt/data/slc_stack_test_subset/results_amster/run_time.log")
if log_path.exists():
    txt = log_path.read_text(errors="ignore")
    m_time = re.search(r"Elapsed \(wall clock\) time.*?:\s*(.+)", txt)
    m_mem  = re.search(r"Maximum resident set size.*?:\s*(\d+)", txt)
    print("Elapsed time:", m_time.group(1).strip() if m_time else "n/a")
    print("Max RSS (KB):", m_mem.group(1) if m_mem else "n/a")
else:
    print("No benchmark log found (run the processing cell first).")
